In [1]:
import pandas as pd
import glob
import matplotlib
import numpy as np
%matplotlib inline
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
import re


In [2]:
def import_folder(folder):
    filenames = glob.glob("data/" + folder + "/*.tsd")

    files = [file.split('\\')[1].split('.')[0] for file in filenames]

    signs = pd.DataFrame()
    for filename in files:
        df = pd.read_table('data/' + folder +'/' + filename + '.tsd', header=None)
        label = filename.split('-')[0]
        ID = filename.rsplit('-', 1)[1]
        df['label'] = label
        df['ID'] = int(ID)
        df.columns = ['x_L','y_L','z_L','roll_L','pitch_L','yaw_L','thumb_L','fore_L','middle_L','ring_L','little_L','x_R','y_R','z_R','roll_R','pitch_R','yaw_R','thumb_R','fore_R','middle_R','ring_R','little_R', 'label', 'ID']
        signs = signs.append(df)
        
    return signs

In [3]:
def temporal_scale(signs):
    IDs = signs['ID'].drop_duplicates().values
    labels = signs['label'].drop_duplicates().values
    signs_new = pd.DataFrame(columns = signs.columns)
    for ID in IDs:
        for sign in labels:

            df = signs[(signs['ID'] == ID) & (signs['label'] == sign)]

            df = df[['x_L', 'y_L', 'z_L', 'roll_L', 'pitch_L', 'yaw_L', 'thumb_L', 'fore_L',
                   'middle_L', 'ring_L', 'little_L', 'x_R', 'y_R', 'z_R', 'roll_R',
                   'pitch_R', 'yaw_R', 'thumb_R', 'fore_R', 'middle_R', 'ring_R',
                   'little_R']]

            df_new = pd.DataFrame(index=np.arange(57), columns = df.columns)

            for column in df:    
                column_resample = np.fft.irfft(np.fft.rfft(df[column]), 57)
                df_new[column] = column_resample

            df_new['label'] = sign
            df_new['ID'] = ID
            df_new['folder'] = folder
            signs_new = signs_new.append(df_new)
            #original = signs_new[['x_L', 'y_L', 'z_L', 'x_R', 'y_R', 'z_R']]
            #scaled = (original - original.min())/(original.max() - original.min())
            #signs_new[['x_L', 'y_L', 'z_L', 'x_R', 'y_R', 'z_R']] = scaled
    return signs_new

In [4]:

root='data'
signs = pd.DataFrame()
signs_all = pd.DataFrame()
signs_all_ts = pd.DataFrame()
dirlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]
for folder in dirlist:
    print(folder)
    signs = import_folder(folder)
    signs_ts = temporal_scale(signs)
    signs_all = signs_all.append(signs)
    signs_all_ts = signs_all_ts.append(signs_ts)

tctodd1
tctodd2
tctodd3
tctodd4
tctodd5
tctodd6
tctodd7
tctodd8
tctodd9


In [5]:
def flatten(df):
    df2 = df
    df['frame'] = df.index + 1
    var = ['x_L', 'y_L', 'z_L', 'roll_L', 'pitch_L', 'yaw_L', 'thumb_L', 'fore_L',
                   'middle_L', 'ring_L', 'little_L', 'x_R', 'y_R', 'z_R', 'roll_R',
                   'pitch_R', 'yaw_R', 'thumb_R', 'fore_R', 'middle_R', 'ring_R',
                   'little_R']
    df = df[['x_L', 'y_L', 'z_L', 'roll_L', 'pitch_L', 'yaw_L', 'thumb_L', 'fore_L',
                   'middle_L', 'ring_L', 'little_L', 'x_R', 'y_R', 'z_R', 'roll_R',
                   'pitch_R', 'yaw_R', 'thumb_R', 'fore_R', 'middle_R', 'ring_R',
                   'little_R', 'label', 'frame']]
    df = df.pivot(columns = 'frame', index = 'label').reset_index()
    df.columns = [[str(e[0]) + str(e[1]) for e in df.columns.values.tolist()]]
    df['ID'] = df2['ID']
    df['folder'] = df2['folder']
    del df2
    return df

In [6]:
IDs = signs['ID'].drop_duplicates().values
labels = signs['label'].drop_duplicates().values

signs_flat = pd.DataFrame()
for folder in dirlist:
    for ID in IDs:
        for sign in labels:
            #print(ID, sign, folder)
            df = signs_all_ts[(signs_all_ts['ID'] == ID) & (signs_all_ts['label'] == sign) & (signs_all_ts['folder'] == folder)]
            signs_flat = signs_flat.append(flatten(df))

C:\Users\charu mulwani\Anaconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
import os
print(os.path.expanduser('~'))

C:\Users\charu mulwani


In [128]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [9]:

# fix random seed for reproducibility
seed = 100
numpy.random.seed(seed)

In [111]:
X_init = signs_flat[signs_flat.columns[2:]]
del X_init['label']
Y_init = signs_flat['label']

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_init)
encoded_Y = encoder.transform(Y_init)

X_train, X_test, y_train, y_test = train_test_split(X_init, encoded_Y, test_size=0.40, random_state=42)



# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(y_train)

X = np.array(X_train)
dummy_y = np.array(dummy_y)

In [129]:
from keras.layers import LSTM
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_shape=(1254,)))
    model.add(Dropout(0.2))
    model.add(Dense(95, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [130]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=5, verbose=1)

In [131]:
estimator.fit(X,dummy_y)

Epoch 1/50
1537/1537 [==============================] - 3s - loss: 3.8484 - acc: 0.1113           

In [137]:
predicted = estimator.predict(np.array(X_test))


 920/1025 [=========================>....] - ETA: 0s  

In [138]:
sum(predicted==y_test)/len(y_test)

0.93658536585365859

In [97]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [98]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 93.17% (2.63%)


In [82]:
for y in Y:
    print(y)



alive
all
answer
boy
building
buy
change_mind_
cold
come
computer_PC_
cost
crazy
danger
deaf
different
draw
drink
eat
exit
flash
forget
girl
give
glove
go
God
happy
head
hear
hello
hot
how
hurry
hurt
I
innocent
is_true_
joke
juice
know
later
lose
love
make
man
maybe
mine
money
more
name
no
Norway
not
paper
pen
please
polite
question
read
ready
research
responsible
right
sad
same
science
share
shop
soon
sorry
spend
stubborn
surprise
take
temper
thank
think
tray
us
voluntary
wait_notyet_
what
when
where
which
who
why
wild
will
write
wrong
yes
you
zero
alive
all
answer
boy
building
buy
change_mind_
cold
come
computer_PC_
cost
crazy
danger
deaf
different
draw
drink
eat
exit
flash
forget
girl
give
glove
go
God
happy
head
hear
hello
hot
how
hurry
hurt
I
innocent
is_true_
joke
juice
know
later
lose
love
make
man
maybe
mine
money
more
name
no
Norway
not
paper
pen
please
polite
question
read
ready
research
responsible
right
sad
same
science
share
shop
soon
sorry
spend
stubborn
surprise
take
tem

In [29]:
signs_flat

,ID,folder,fore_L1,fore_L10,fore_L11,fore_L12,fore_L13,fore_L14,fore_L15,fore_L16,...,z_R52,z_R53,z_R54,z_R55,z_R56,z_R57,z_R6,z_R7,z_R8,z_R9
0,1.0,tctodd1,3.507470e-02,2.888556e-03,1.248671e-02,1.755758e-02,1.438340e-02,9.732387e-03,1.162745e-02,1.506117e-02,...,0.003290,-0.000097,0.003122,0.001192,0.004100,0.001781,0.114482,0.068755,0.029946,0.054653
0,1.0,tctodd1,1.047991e-02,1.962489e-02,3.178795e-02,3.974623e-02,4.366870e-02,4.912068e-02,5.196587e-02,4.467303e-02,...,-0.008486,-0.008672,-0.008162,-0.008501,-0.008917,-0.008166,-0.212868,-0.237275,-0.216860,-0.164582
0,1.0,tctodd1,2.289398e-01,1.089710e+00,1.084585e+00,1.092367e+00,1.080822e+00,1.098455e+00,1.069065e+00,1.128868e+00,...,-0.046039,-0.044744,-0.044232,-0.041629,-0.045013,-0.044093,0.100942,0.113385,0.063589,-0.009982
0,1.0,tctodd1,5.119614e-02,2.701419e-02,2.078903e-02,3.030439e-02,2.947683e-02,3.011796e-02,2.991292e-02,2.805684e-02,...,-0.068772,-0.065889,-0.068437,-0.065503,-0.069751,-0.064823,0.143884,0.161429,0.139829,0.123353
0,1.0,tctodd1,5.602339e-02,2.140258e-01,5.228006e-01,9.659331e-01,6.941850e-01,3.340363e-01,2.596844e-01,2.241408e-01,...,0.000097,0.000847,-0.003912,-0.002249,-0.005493,-0.001332,-0.061935,-0.081762,-0.057542,-0.086377
0,1.0,tctodd1,6.434386e-04,2.012085e-02,1.915952e-02,1.316210e-02,1.633990e-03,1.247796e-02,1.537083e-02,-3.361528e-03,...,-0.043818,-0.042885,-0.043413,-0.043624,-0.043707,-0.043445,-0.065687,-0.064017,-0.058626,-0.066193
0,1.0,tctodd1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-0.009205,-0.010772,-0.009618,-0.009805,-0.008953,-0.008538,0.059827,0.081631,0.156303,0.168927
0,1.0,tctodd1,1.171672e+00,1.428776e+00,1.375307e+00,1.436716e+00,1.362126e+00,1.437651e+00,4.779017e-01,-7.146885e-02,...,-0.081745,-0.085810,-0.079920,-0.088530,-0.078816,-0.095343,0.227759,0.191238,0.202215,0.192107
0,1.0,tctodd1,1.548596e-03,1.440104e-03,-2.446905e-04,1.511797e-04,-1.317024e-04,1.354988e-04,-1.511153e-04,1.745212e-04,...,-0.005054,-0.008367,-0.005651,-0.009098,-0.004409,-0.010148,-0.212464,-0.204564,-0.147249,-0.047366
0,1.0,tctodd1,1.283534e-01,6.232310e-02,2.468620e-02,1.370890e-02,5.059890e-02,9.794307e-02,1.135795e-01,1.289836e-01,...,-0.023777,-0.023332,-0.022362,-0.025360,-0.020360,-0.030723,0.105299,0.155827,0.115091,0.099130
